# **Building a Power BI SQL Agent monitoring template**

Power BI can be leveraged to bring insights to all manner of business and technical problems.

This example covers creating a model for monitoring SQL Agent job activity.
Other examples would be Database Schema and index reports


## Considerations:
What "dimension" tables are needed/common across possible data sets?

What to use as Keys for dimensions. 

    Must be single column. 
    Must be unique, preferably numeric.


## Data sets

Create data sets for each of the following:

- Alerts
- Calendar - M function 
- Time of Day - M function
- SQL Agent Settings
- HistoryRange
- Job 
- Job categories
- job clash
- Job Schedules
- Job Status
- Job Steps
- Job Step History
- Schedules

## Parameters

For the model to serve as a template, we need to specify a couple parameters in the query designer that will be used by most of the subsequent queries to pull data from a server specified in the parameter.

> Parameter: SQLServerInstance 

* Description: Instance name to connect to in order to read SQL Agent job data

* Type: Text

* Current Value: LocalHost

> Parameter: Culture

* Description: ISO Culture code, used to derive calendar attribute name language. Normally, leavethis blank. See https://lonewolfonline.net/list-net-culture-country-codes/ for a complete list

* Type: Text

* Current Value: 



### Alerts
Leverage data from [dbo].[sysalerts] table in the [MSDB] database

In [0]:
SELECT [sa].[id]
     , [sa].[name] AS [Alert Name]
     , [sa].[event_source] AS [Event Source]
     , [sa].[event_category_id]
     , [sa].[event_id]
     , [sa].[message_id]
	 , [sa].[category_id]
     , [sa].[job_id]
     , [sa].[Severity]
     , [sa].[Enabled]
     , [sa].[delay_between_responses] AS [Delay Between Responses]
     , [dc1].[last_occurrence_datetime] AS [Last Occurence]
     , [dc1].[last_occurrence_date] AS [Last Occurence Date]
     , [dc1].[last_occurrence_time] AS [Last Occurence Time]
     , [dc1].[last_response_datetime] AS [Last Response]
     , [dc1].[last_response_date] AS [Last Response Date]
     , [dc1].[last_response_time] AS [Last Response Time]
     , [sa].[notification_message] AS [Notification Message]
     , [sa].[include_event_description] AS [Include Event Description]
     , [sa].[database_name] AS [database Name]
     , [sa].[event_description_keyword] AS [Event Description Keyword]
     , [sa].[occurrence_count] AS [Occurences]
     , [dc1].[count_reset_datetime] AS [Count Reset]
     , [dc1].[count_reset_date] AS [Count Reset Date]
     , [dc1].[count_reset_time] AS [Count Reset Time]
     , [sa].[has_notification] AS [Has Notification]
     , [sa].[Flags]
     , [sa].[performance_condition] AS [Performance Condition]
FROM [dbo].[sysalerts] AS [sa]
     OUTER APPLY
     (
         SELECT CAST(CONVERT(NVARCHAR(4), NULLIF([sa].[last_occurrence_date], 0) / 10000)
                     + N'-'
                     + CONVERT(NVARCHAR(2), ( [sa].[last_occurrence_date] % 10000 ) / 100)
                     + N'-'
                     + CONVERT(NVARCHAR(2), [sa].[last_occurrence_date] % 100)
                     + N' '
                     + CONVERT(NVARCHAR(2), [sa].[last_occurrence_time] / 10000)
                     + N':'
                     + CONVERT(NVARCHAR(2), ( [sa].[last_occurrence_time] % 10000 ) / 100)
                     + N':'
                     + CONVERT(NVARCHAR(2), [sa].[last_occurrence_time] % 100) AS DATETIME) AS [last_occurrence_datetime]
              , CAST(CONVERT(NVARCHAR(4), NULLIF([sa].[last_occurrence_date], 0) / 10000)
                     + N'-'
                     + CONVERT(NVARCHAR(2), ( [sa].[last_occurrence_date] % 10000 ) / 100)
                     + N'-'
                     + CONVERT(NVARCHAR(2), [sa].[last_occurrence_date] % 100) AS DATE) AS [last_occurrence_date]
              , CAST(CONVERT(NVARCHAR(2), NULLIF([sa].[last_occurrence_time], 0) / 10000)
                     + N':'
                     + CONVERT(NVARCHAR(2), ( [sa].[last_occurrence_time] % 10000 ) / 100)
                     + N':'
                     + CONVERT(NVARCHAR(2), [sa].[last_occurrence_time] % 100) AS TIME) AS [last_occurrence_Time]
              , CAST(CONVERT(NVARCHAR(4), NULLIF([sa].[last_response_date], 0) / 10000)
                     + N'-'
                     + CONVERT(NVARCHAR(2), ( [sa].[last_response_date] % 10000 ) / 100)
                     + N'-'
                     + CONVERT(NVARCHAR(2), [sa].[last_response_date] % 100)
                     + N' '
                     + CONVERT(NVARCHAR(2), [sa].[last_response_time] / 10000)
                     + N':'
                     + CONVERT(NVARCHAR(2), ( [sa].[last_response_time] % 10000 ) / 100)
                     + N':'
                     + CONVERT(NVARCHAR(2), [sa].[last_response_time] % 100) AS DATETIME) AS [last_response_datetime]
              , CAST(CONVERT(NVARCHAR(4), NULLIF([sa].[last_response_date], 0) / 10000)
                     + N'-'
                     + CONVERT(NVARCHAR(2), ( [sa].[last_response_date] % 10000 ) / 100)
                     + N'-'
                     + CONVERT(NVARCHAR(2), [sa].[last_response_date] % 100) AS DATE) AS [last_response_date]
              , CAST(CONVERT(NVARCHAR(2), NULLIF([sa].[last_response_time], 0) / 10000)
                     + N':'
                     + CONVERT(NVARCHAR(2), ( [sa].[last_response_time] % 10000 ) / 100)
                     + N':'
                     + CONVERT(NVARCHAR(2), [sa].[last_response_time] % 100) AS TIME) AS [last_response_Time]
              , CAST(CONVERT(NVARCHAR(4), NULLIF([sa].[count_reset_date], 0) / 10000)
                     + N'-'
                     + CONVERT(NVARCHAR(2), ( [sa].[count_reset_date] % 10000 ) / 100)
                     + N'-'
                     + CONVERT(NVARCHAR(2), [sa].[count_reset_date] % 100)
                     + N' '
                     + CONVERT(NVARCHAR(2), [sa].[count_reset_time] / 10000)
                     + N':'
                     + CONVERT(NVARCHAR(2), ( [sa].[count_reset_time] % 10000 ) / 100)
                     + N':'
                     + CONVERT(NVARCHAR(2), [sa].[count_reset_time] % 100) AS DATETIME) AS [count_reset_datetime]
              , CAST(CONVERT(NVARCHAR(4), NULLIF([sa].[count_reset_date], 0) / 10000)
                     + N'-'
                     + CONVERT(NVARCHAR(2), ( [sa].[count_reset_date] % 10000 ) / 100)
                     + N'-'
                     + CONVERT(NVARCHAR(2), [sa].[count_reset_date] % 100) AS DATE) AS [count_reset_date]
              , CAST(CONVERT(NVARCHAR(2), NULLIF([sa].[count_reset_time], 0) / 10000)
                     + N':'
                     + CONVERT(NVARCHAR(2), ( [sa].[count_reset_time] % 10000 ) / 100)
                     + N':'
                     + CONVERT(NVARCHAR(2), [sa].[count_reset_time] % 100) AS TIME) AS [count_reset_Time]
     ) AS [dc1];

### Calendar - M function 

Create a blank M query and add the code below and name it fnCalendar

In [0]:
//Create Date Dimension
let
    Source = (StartDate as date, EndDate as date, optional Culture as nullable text)=>

let
    //Capture the date range from the parameters
    StartDate = #date(Date.Year(StartDate), Date.Month(StartDate), 
    Date.Day(StartDate)),
    EndDate = #date(Date.Year(EndDate), Date.Month(EndDate), 
    Date.Day(EndDate)),

    //Get the number of dates that will be required for the table
    GetDateCount = Duration.Days(EndDate - StartDate),

    //Take the count of dates and turn it into a list of dates
    GetDateList = List.Dates(StartDate, GetDateCount, 
    #duration(1,0,0,0)),

    //Convert the list into a table
    DateListToTable = Table.FromList(GetDateList, 
    Splitter.SplitByNothing(), {"Date"}, null, ExtraValues.Error),

    //Add sequentialDayNumber
    SequentialDayNumber = Table.AddIndexColumn(DateListToTable , "SequentialDayNumber", 1, 1),

    //Create various date attributes from the date column
    //Add Year Column
    YearNumber = Table.AddColumn(SequentialDayNumber, "YearNumber", 
    each Date.Year([Date]), Int64.Type),

    //Add End of Year Column
    YearEnds = Table.AddColumn(YearNumber, "YearEnds", 
    each Date.EndOfYear([Date]), type date),

    //Add QuarterNumber Column
    QuarterNumber = Table.AddColumn(YearEnds, "QuarterNumber", 
    each Date.QuarterOfYear([Date]), Int64.Type),

    //Add Quarter Column
    Quarter = Table.AddColumn(QuarterNumber , "Quarter", 
    each "Q" & Number.ToText(Date.QuarterOfYear([Date]))),

    //Add YearQuarter Column
    YearQuarter = Table.AddColumn(Quarter , "YearQuarter", 
    each Number.ToText(Date.Year([Date])) &" Q" & Number.ToText(Date.QuarterOfYear([Date]))),

    //Add YearQuarterNumber Column
    YearQuarterNumber = Table.AddColumn(YearQuarter, "YearQuarterNumber", 
    each Number.ToText(Date.Year([Date])) & "0" & Number.ToText(Date.QuarterOfYear([Date]))),

    //Add Week Number Column
    WeekNumber= Table.AddColumn(YearQuarterNumber , "WeekNumber", 
    each Date.WeekOfYear([Date]), Int64.Type),

    //Add Week Beginning
    WeekBeginning= Table.AddColumn(WeekNumber, "Week Beginning", 
    each Date.StartOfWeek([Date]), type date),

    //Add Week Ending
    WeekEnding= Table.AddColumn(WeekBeginning, "Week Ending", 
    each Date.EndOfWeek([Date]), type date),

    //Add Month Number Column
    MonthNumber = Table.AddColumn(WeekEnding, "MonthNumber", 
    each Date.Month([Date]), Int64.Type),

    //Add YearMonthNumber Column
    YearMonthNumber = Table.AddColumn(MonthNumber, "YearMonthNumber", 
    each Date.ToText([Date], "yyyyMM")),


    //Add YearMonth Column
    YearMonth = Table.AddColumn(YearMonthNumber, "YearMonth", 
    each Date.ToText([Date], "yyyy-MM")),

    //Add Month Name Column
    MonthName = Table.AddColumn(YearMonth, "Month", 
    each Date.ToText([Date],"MMMM", Culture)),

    //Add Month Name Column
    MonthAbbv = Table.AddColumn(MonthName, "Month Abbv", 
    each Date.ToText([Date],"MMM", Culture)),

    //Add Days in Month Column
    MonthDays = Table.AddColumn(MonthAbbv, "MonthDays", 
    each Date.DaysInMonth([Date]), Int64.Type),

    MonthDayNumber = Table.AddColumn(MonthDays, "MonthDayNumber", 
    each Date.Day([Date]), Int64.Type),

    //Add Day of Week Column
    DayOfWeek = Table.AddColumn(MonthDayNumber, "Day of Week", 
    each Date.ToText([Date],"dddd", Culture)),

   //Add Day Abbv of Week Column
    DayAbbv = Table.AddColumn(DayOfWeek, "Day Abbv", 
    each Date.ToText([Date],"ddd", Culture)),

    //Add Day of Week Number sorting Column
    DayOfWeekNumber = Table.AddColumn(DayAbbv , "DOW Number", 
    each Date.DayOfWeek([Date])+1, Int64.Type),

    //Add Date datatype key
    DateKey = Table.AddColumn(DayOfWeekNumber , "DateKey",
    each Date.ToText([Date], "yyyyMMdd")),

    //Add DateTime datatype key
    DateTimeKey = Table.AddColumn(DateKey , "FullDateKey",
    each DateTime.From([Date]), type datetime),

    //Add a column that returns true if the date on rows is the current date
    IsToday = Table.AddColumn(DateTimeKey, "IsToday", each Date.IsInCurrentDay([Date]), type logical),

    //Add a column that returns true if the date on rows is the current date
    IsThisYear = Table.AddColumn(IsToday, "IsThisYear", each Date.IsInCurrentYear([Date]), type logical),

    //Add a column that returns true if the date on rows is the current date
    IsThisMonth = Table.AddColumn(IsThisYear, "IsThisMonth", each Date.IsInCurrentMonth([Date]), type logical),

    //Add a column that returns true if the date a future date
    Isfuture = Table.AddColumn(IsThisMonth, "IsFuture", each Date.IsInNextNDays([Date],999999), type logical),

    //transform data types. The last two generated errors when trying to convert inline above.
    TransformTypes = Table.TransformColumnTypes(Isfuture , 
		{
		{"Date", type date}
                ,{"DateKey", Int64.Type}
                ,{"YearMonthNumber", Int64.Type}
		})

in
    TransformTypes
in
    Source

### Calendar Data Set M call

Create a blank M query and add the code below to invoke fnCalendar

In [0]:
let
    BeginDate = List.Min(Table.Column(HistoryRange,"begin_date")),
    EndDate = List.Min(Table.Column(HistoryRange,"end_date")),
    Source = #"fnCalendar"(BeginDate, EndDate),
    #"Renamed Columns" = Table.RenameColumns(Source,{{"IsToday", "Is Today"}, 
      {"IsFuture", "Is Future"}, {"WeekNumber", "Week Number"}, {"YearEnds", "Year Ends"}, {"MonthDayNumber", "Month Day Number"}, {"MonthDays", "Month Days"}, {"MonthNumber", "Month Number"}, {"QuarterNumber", "Quarter Number"}, {"YearMonth", "Year Month"}, {"YearNumber", "Year Number"}, {"YearQuarter", "Year Quarter"}, {"YearQuarterNumber", "Year Quarter Number"}, {"IsThisMonth", "Is This Month"}, {"IsThisYear", "Is This Year"}})
in
    #"Renamed Columns"

### Time of Day - M function

Add a blank M Query, add the code below and rename it to "Time of Day"

In [0]:
let
    Source = List.Times(#time(0,0,0) , 86400, #duration(0,0,0,1)),
    ConvertToTable = Table.FromList(Source, Splitter.SplitByNothing(), {"Time of Day"}, null, ExtraValues.Error),
    //HHMMSS formtat "TimeKey"
    CreateTimeKey = Table.AddColumn(ConvertToTable, "TimeKey", each Time.ToText([Time of Day], "HHmmss")),
    ChangeTimeKeyType = Table.TransformColumnTypes(CreateTimeKey,{{"TimeKey", Int64.Type}}),
    //Raw Hour (0-23), Minute (0-59) & second (0-59) values
    Hour = Table.AddColumn(ChangeTimeKeyType, "Hour", each Time.Hour([Time of Day]), Int64.Type),
    Minute = Table.AddColumn(Hour, "Minute", each Time.Minute([Time of Day]), Int64.Type),
    Second = Table.AddColumn(Minute, "Second", each Time.Second([Time of Day]), Int64.Type),

    //Sequential keys for sorting display values
    HourKey = Table.AddColumn(Second, "HourKey", each Time.Hour([Time of Day]), Int64.Type),
    MinuteKey = Table.AddColumn(HourKey, "MinuteKey", each ([Hour] * 60) + [Minute] , Int64.Type),
    SecondKey = Table.AddIndexColumn(MinuteKey, "SecondKey", 1, 1),
    
    //Display values for hierarchy levels
    HourOfDay = Table.AddColumn(SecondKey,"Hour of Day", each Time.ToText(#time([Hour],0,0))),
    MinuteOfDay = Table.AddColumn(HourOfDay,"Minute of Day", each Time.ToText(#time([Hour],[Minute],0))),
    MilitaryHour = Table.AddColumn(MinuteOfDay, "Military Hour", each Time.ToText(#time([Hour],[Minute],[Second]), "HH:00:00")),
    MilitaryMinute = Table.AddColumn(MilitaryHour, "Military Minute", each Time.ToText(#time([Hour],[Minute],[Second]), "HH:mm:00")),
    MilitaryTime = Table.AddColumn(MilitaryMinute, "Military Time", each Time.ToText(#time([Hour],[Minute],[Second]), "HH:mm:ss")),

    //standard work shifts (00:00 - 7:59, 8:00 - 16:59, 17:00 - 23:59)
    AddShift = Table.AddColumn(MilitaryTime, "Shift", 
        each if [TimeKey] < 70000 then 3 else if [TimeKey] < 153000 then 1 else 2 ),
    AddShiftName = Table.AddColumn(AddShift, "Shift Name", 
        each if [TimeKey] < 70000 then "Shift 3" else if [TimeKey] < 153000 then "Shift 1" else "Shift 2" ),
    #"Filtered Rows" = Table.SelectRows(AddShiftName, each true)
in
    #"Filtered Rows"

### SQL Agent Settings


In [0]:
DECLARE @max_total_rows INT = 0;
DECLARE @max_rows_per_job INT = 0;
DECLARE @Version NVARCHAR(255);

EXECUTE [master].[dbo].[xp_instance_regread] 
        N'HKEY_LOCAL_MACHINE'
      , N'SOFTWARE\Microsoft\MSSQLServer\SQLServerAgent'
      , N'JobHistoryMaxRows'
      , @max_total_rows OUTPUT
      , N'no_output';

EXECUTE [master].[dbo].[xp_instance_regread] 
        N'HKEY_LOCAL_MACHINE'
      , N'SOFTWARE\Microsoft\MSSQLServer\SQLServerAgent'
      , N'JobHistoryMaxRowsPerJob'
      , @max_rows_per_job OUTPUT
      , N'no_output';
if exists (select * from sys.tables where name = 'msdb_version')
	begin
		SELECT @Version = [version_string]
		FROM [dbo].[msdb_version]
		WHERE [id] =
			  (
				  SELECT MAX([id])
				  FROM [dbo].[msdb_version]  )
	end
	else
	begin
		SELECT @Version = 'Time to upgrade!'
	end
SELECT @max_total_rows AS [MaxJobHistoryTableRows]
    , @max_rows_per_job AS [MaxJobHistoryRows]
    , @Version AS [MSDB Version]


### History Range

Get the range of dates in the SQL Agent History table

In [0]:
WITH CTE_Range
AS (SELECT MIN([run_date]) AS [begin_date]
        , MAX([run_date]) AS [end_date]
    FROM [dbo].[sysjobhistory] WITH(NOLOCK)
    WHERE [step_id] = 0
	)
SELECT @@SERVERNAME AS [Server]
    , CAST(CONVERT(NVARCHAR(4), [begin_date] / 10000)
            + N'-'
            + CONVERT(NVARCHAR(2), ( [begin_date] % 10000 ) / 100)
            + N'-'
            + CONVERT(NVARCHAR(2), [begin_date] % 100) AS DATE) AS [begin_date]
    , DATEADD(DAY, 1, CAST(CONVERT(NVARCHAR(4), [end_date] / 10000)
            + N'-'
            + CONVERT(NVARCHAR(2), ( [end_date] % 10000 ) / 100)
            + N'-'
            + CONVERT(NVARCHAR(2), [end_date] % 100) AS DATE)
			) AS [end_date]
FROM [CTE_Range];

M Script to set date data types on begin and end date columns:

In [0]:
let
    Source = Sql.Database(SQLServerInstance, "msdb", [Query="WITH CTE_Range#(lf)AS (SELECT MIN([run_date]) AS [begin_date]#(lf)        , MAX([run_date]) AS [end_date]#(lf)    FROM [dbo].[sysjobhistory] WITH(NOLOCK)#(lf)    WHERE [step_id] = 0#(lf)#(tab))#(lf)SELECT @@SERVERNAME AS [Server]#(lf)    , CAST(CONVERT(NVARCHAR(4), [begin_date] / 10000)#(lf)            + N'-'#(lf)            + CONVERT(NVARCHAR(2), ( [begin_date] % 10000 ) / 100)#(lf)            + N'-'#(lf)            + CONVERT(NVARCHAR(2), [begin_date] % 100) AS DATE) AS [begin_date]#(lf)    , DATEADD(DAY, 1, CAST(CONVERT(NVARCHAR(4), [end_date] / 10000)#(lf)            + N'-'#(lf)            + CONVERT(NVARCHAR(2), ( [end_date] % 10000 ) / 100)#(lf)            + N'-'#(lf)            + CONVERT(NVARCHAR(2), [end_date] % 100) AS DATE)#(lf)#(tab)#(tab)#(tab)) AS [end_date]#(lf)FROM [CTE_Range];", CreateNavigationProperties=false]),
    #"Changed Type" = Table.TransformColumnTypes(Source,{{"begin_date", type date}, {"end_date", type date}})
in
    #"Changed Type"

### Job 

**SQL Source**

In [0]:
SELECT [J].[job_id]
     , [j].[category_id]
	 , [c].[name] as [Category]
     , [j].[name] AS [Job Name]
     , ROW_NUMBER() OVER (ORDER BY [j].[name]) AS job_order
     , [j].[Enabled]
     , ISNULL([sp].[name], 'Unknown') AS [Job Owner]
     , [DC1].[last_run_date] AS [Last Run Date]
     , [DC1].[last_run_time] AS [Last Run Time]
     , [DC1].[last_run_datetime] AS [Last Run Datetime]
     , ISNULL([ss].[last_run_outcome], 0) AS [Last Run Outcome]
     , [ss].[last_outcome_message] AS [Last Outcome Message]
     , ( ( [ss].[last_run_duration] / 10000 ) * 3600 )
       + ( ( ( [ss].[last_run_duration] % 10000 ) / 100 ) * 60 )
       + ( [ss].[last_run_duration] % 100 ) AS [Last Run Duration]
FROM [dbo].[sysjobs] AS [j] WITH(NOLOCK)
LEFT OUTER JOIN [dbo].[syscategories] AS [c] WITH (NOLOCK)
ON [j].[category_id] = [c].[category_id]
      JOIN [sys].[server_principals] AS [sp] WITH(NOLOCK)
     ON [j].[owner_sid] = [sp].[sid]
     LEFT OUTER JOIN [dbo].[sysjobservers] AS [ss] WITH(NOLOCK)
     ON [j].[job_id] = [ss].[job_id]
     OUTER APPLY
     (
         SELECT CAST(CONVERT(NVARCHAR(4), NULLIF([ss].[last_run_date], 0) / 10000)
                     + N'-'
                     + CONVERT(NVARCHAR(2), ( [ss].[last_run_date] % 10000 ) / 100)
                     + N'-'
                     + CONVERT(NVARCHAR(2), [ss].[last_run_date] % 100)
                     + N' '
                     + CONVERT(NVARCHAR(2), [ss].[last_run_time] / 10000)
                     + N':'
                     + CONVERT(NVARCHAR(2), ( [ss].[last_run_time] % 10000 ) / 100)
                     + N':'
                     + CONVERT(NVARCHAR(2), [ss].[last_run_time] % 100) AS DATETIME) AS [last_run_datetime]
              , CAST(CONVERT(NVARCHAR(4), NULLIF([ss].[last_run_date], 0) / 10000)
                     + N'-'
                     + CONVERT(NVARCHAR(2), ( [ss].[last_run_date] % 10000 ) / 100)
                     + N'-'
                     + CONVERT(NVARCHAR(2), [ss].[last_run_date] % 100) AS DATE) AS [last_run_date]
              , CAST(CONVERT(NVARCHAR(2), NULLIF([ss].[last_run_time], 0) / 10000)
                     + N':'
                     + CONVERT(NVARCHAR(2), ( [ss].[last_run_time] % 10000 ) / 100)
                     + N':'
                     + CONVERT(NVARCHAR(2), [ss].[last_run_time] % 100) AS TIME) AS [last_run_Time]
     ) AS [dc1];


**M Script**

Create the **Job** query using the following M script, which contains the above query.

In [0]:
let
    Source = Sql.Database(SQLServerInstance, "msdb", [Query="SELECT [J].[job_id]#(lf)     , [j].[category_id]#(lf)#(tab) , [c].[name] as [Category]#(lf)     , [j].[name] AS [Job Name]#(lf)     , ROW_NUMBER() OVER (ORDER BY [j].[name]) AS job_order#(lf)     , [j].[Enabled]#(lf)     , ISNULL([sp].[name], 'Unknown') AS [Job Owner]#(lf)     , [DC1].[last_run_date] AS [Last Run Date]#(lf)     , [DC1].[last_run_time] AS [Last Run Time]#(lf)     , [DC1].[last_run_datetime] AS [Last Run Datetime]#(lf)     , ISNULL([ss].[last_run_outcome], 0) AS [Last Run Outcome]#(lf)     , [ss].[last_outcome_message] AS [Last Outcome Message]#(lf)     , ( ( [ss].[last_run_duration] / 10000 ) * 3600 )#(lf)       + ( ( ( [ss].[last_run_duration] % 10000 ) / 100 ) * 60 )#(lf)       + ( [ss].[last_run_duration] % 100 ) AS [Last Run Duration]#(lf)FROM [dbo].[sysjobs] AS [j] WITH(NOLOCK)#(lf)LEFT OUTER JOIN [dbo].[syscategories] AS [c] WITH (NOLOCK)#(lf)ON [j].[category_id] = [c].[category_id]#(lf)      JOIN [sys].[server_principals] AS [sp] WITH(NOLOCK)#(lf)     ON [j].[owner_sid] = [sp].[sid]#(lf)     LEFT OUTER JOIN [dbo].[sysjobservers] AS [ss] WITH(NOLOCK)#(lf)     ON [j].[job_id] = [ss].[job_id]#(lf)     OUTER APPLY#(lf)     (#(lf)         SELECT CAST(CONVERT(NVARCHAR(4), NULLIF([ss].[last_run_date], 0) / 10000)#(lf)                     + N'-'#(lf)                     + CONVERT(NVARCHAR(2), ( [ss].[last_run_date] % 10000 ) / 100)#(lf)                     + N'-'#(lf)                     + CONVERT(NVARCHAR(2), [ss].[last_run_date] % 100)#(lf)                     + N' '#(lf)                     + CONVERT(NVARCHAR(2), [ss].[last_run_time] / 10000)#(lf)                     + N':'#(lf)                     + CONVERT(NVARCHAR(2), ( [ss].[last_run_time] % 10000 ) / 100)#(lf)                     + N':'#(lf)                     + CONVERT(NVARCHAR(2), [ss].[last_run_time] % 100) AS DATETIME) AS [last_run_datetime]#(lf)              , CAST(CONVERT(NVARCHAR(4), NULLIF([ss].[last_run_date], 0) / 10000)#(lf)                     + N'-'#(lf)                     + CONVERT(NVARCHAR(2), ( [ss].[last_run_date] % 10000 ) / 100)#(lf)                     + N'-'#(lf)                     + CONVERT(NVARCHAR(2), [ss].[last_run_date] % 100) AS DATE) AS [last_run_date]#(lf)              , CAST(CONVERT(NVARCHAR(2), NULLIF([ss].[last_run_time], 0) / 10000)#(lf)                     + N':'#(lf)                     + CONVERT(NVARCHAR(2), ( [ss].[last_run_time] % 10000 ) / 100)#(lf)                     + N':'#(lf)                     + CONVERT(NVARCHAR(2), [ss].[last_run_time] % 100) AS TIME) AS [last_run_Time]#(lf)     ) AS [dc1];#(lf)", CreateNavigationProperties=false]),
    #"Changed Type" = Table.TransformColumnTypes(Source,{{"Enabled", type logical}, {"Last Run Outcome", type logical}}),
    #"Duplicated Column" = Table.DuplicateColumn(#"Changed Type", "Job Name", "Job Name - Copy"),
    #"Added Custom" = Table.AddColumn(#"Duplicated Column", "Custom", each if [Category] = "Report Server" then "Report Server" else [Job Name]),
    #"Split Column by Delimiter" = Table.SplitColumn(#"Added Custom", "Custom", Splitter.SplitTextByEachDelimiter({"_"}, QuoteStyle.Csv, false), {"Custom.1", "Custom.2"}),
    #"Changed Type1" = Table.TransformColumnTypes(#"Split Column by Delimiter",{{"Custom.1", type text}, {"Custom.2", type text}}),
    #"Removed Columns" = Table.RemoveColumns(#"Changed Type1",{"Custom.2"}),
    #"Split Column by Delimiter1" = Table.SplitColumn(#"Removed Columns", "Custom.1", Splitter.SplitTextByEachDelimiter({"-"}, QuoteStyle.Csv, false), {"Custom.1.1", "Custom.1.2"}),
    #"Changed Type2" = Table.TransformColumnTypes(#"Split Column by Delimiter1",{{"Custom.1.1", type text}, {"Custom.1.2", type text}}),
    #"Removed Columns1" = Table.RemoveColumns(#"Changed Type2",{"Custom.1.2"}),
    #"Renamed Columns" = Table.RenameColumns(#"Removed Columns1",{{"Custom.1.1", "Job Prefix"}}),
    #"Removed Columns2" = Table.RemoveColumns(#"Renamed Columns",{"Job Name - Copy"}),
    #"Renamed Columns1" = Table.RenameColumns(#"Removed Columns2",{{"Last Run Duration", "Last_Run_Duration"}})
in
    #"Renamed Columns1"

### Job categories


In [0]:
SELECT [category_id]
     , [name] AS [Category]
     , CASE [category_class]
           WHEN 1
           THEN CASE 
				WHEN LEFT([name], 4) = 'repl'
				THEN 'Replication'
				WHEN [name] = 'Report Server'
				Then 'Reports'
				ELSE 'Job'
				END
           WHEN 2
           THEN 'Alert'
           WHEN 3
           THEN 'Operator'
       END AS [Category Class]
     , CASE [category_type]
           WHEN 1
           THEN 'Local'
           WHEN 2
           THEN 'Multiserver'
           WHEN 3
           THEN 'None'
       END AS [Category Type]
FROM [dbo].[syscategories]
order by 1

### Job Clash
This query determines the number of jobs executing concurrently at any given second of the day.

In [0]:
WITH SecondsCTE
     AS
     /* Row Per second of the day */
     (SELECT 0 AS [SecondKey] /*Sequential second key*/
           , CAST('00:00:00' AS TIME) AS [Time]
           , CAST(0 AS INT) AS [TimeKey] /* HHMMSS time key */
      UNION ALL
      SELECT [SecondKey] + 1
           , CAST(DATEADD([SECOND], 1, [Time]) AS TIME) AS [Time]
           , CAST(replace(CONVERT(VARCHAR(8), CAST(DATEADD([SECOND], 1, [Time]) AS TIME), 13), ':', '') AS INT) AS [TimeKey]
      FROM [SecondsCTE]
      WHERE ( [SecondKey] + 1 ) < ( 60 * 60 * 24 ) /* End Recursion */
     ),
cte_history
AS (SELECT [job_id]
,[run_date]
,[run_time]
        , [Start_Date]
        , [Start_Time]
        , CAST([End_DateTime] AS DATE) AS [End_Date]
        , CAST([End_DateTime] AS TIME) AS [End_Time]
        , 1 AS [Duration]
    FROM [dbo].[sysjobhistory] WITH(NOLOCK)
    CROSS APPLY
      (
        SELECT CAST(CONVERT(NVARCHAR(4), [run_date] / 10000)
                    + N'-'
                    + CONVERT(NVARCHAR(2), ( [run_date] % 10000 ) / 100)
                    + N'-'
                    + CONVERT(NVARCHAR(2), [run_date] % 100) AS DATE) 
			  AS [Start_Date]
            , CAST(CONVERT(NVARCHAR(2), [run_time] / 10000)
                    + N':'
                    + CONVERT(NVARCHAR(2), ( [run_time] % 10000 ) / 100)
                    + N':'
                    + CONVERT(NVARCHAR(2), [run_time] % 100) AS TIME) 
			  AS [start_time]
            , DATEADD([SECOND]
						, CASE WHEN[run_duration] < 60
							THEN [run_duration]
							ELSE ( ( [run_duration] / 10000 * 3600 + ( [run_duration] / 100 ) % 100 * 60 + [run_duration] % 100 ) )
							END
						, CAST(CONVERT(NVARCHAR(4), [run_date] / 10000)
                                + N'-' + CONVERT(NVARCHAR(2), ( [run_date] % 10000 ) / 100)
                                + N'-' + CONVERT(NVARCHAR(2), [run_date] % 100)
                                + N' ' + CONVERT(NVARCHAR(2), [run_time] / 10000)
                                + N':' + CONVERT(NVARCHAR(2), ( [run_time] % 10000 ) / 100)
                                + N':' + CONVERT(NVARCHAR(2), [run_time] % 100) AS DATETIME)) 
			  AS [End_DateTime]
      ) AS [_]
         WHERE [step_id] = 0
               AND [Start_Date] = CAST(DATEADD(DAY, -1, GETDATE()) AS DATE)

         /*get clash data for prior complete day only.*/
   )

SELECT [S].[TimeKey] AS [TimeKey]
    , SUM(H.Duration) AS [Running]
FROM [SecondsCTE] AS [S]
CROSS JOIN [cte_history] AS [H]
/* Cartesian result, filtered by the start/end range of the job*/
WHERE [s].[Time] >= [h].[Start_time]
  AND [s].[Time] <= [h].[End_time]
GROUP BY [S].[TimeKey] 
OPTION(MAXRECURSION 0);

### Job Schedules


In [0]:
SELECT [schedule_id]
	,[job_id]
	,[next_run_date] as [next_run_datekey]
	,CAST(CONVERT(NVARCHAR(4), NULLIF([next_run_date], 0) / 10000)
		+ N'-'+ CONVERT(NVARCHAR(2), ( [next_run_date] % 10000 ) / 100)
		+ N'-'+ CONVERT(NVARCHAR(2), [next_run_date] % 100)
		+ N' '+ CONVERT(NVARCHAR(2), [next_run_time] / 10000)
		+ N':'+ CONVERT(NVARCHAR(2), ( [next_run_time] % 10000 ) / 100)
		+ N':'+ CONVERT(NVARCHAR(2), [next_run_time] % 100) AS DATETIME) AS [next_run_datetime]
	, CAST(CONVERT(NVARCHAR(4), NULLIF([next_run_date], 0) / 10000)
		+ N'-'+ CONVERT(NVARCHAR(2), ( [next_run_date] % 10000 ) / 100)
		+ N'-'+ CONVERT(NVARCHAR(2), [next_run_date] % 100) AS DATE) AS [next_run_date]
	, CAST(CONVERT(NVARCHAR(2), NULLIF([next_run_time], 0) / 10000)
		+ N':'+ CONVERT(NVARCHAR(2), ( [next_run_time] % 10000 ) / 100)
		+ N':'+ CONVERT(NVARCHAR(2), [next_run_time] % 100) AS TIME) AS [next_run_Time]
FROM [dbo].[sysjobschedules] AS [jobsched] WITH (NOLOCK)

### Job Status

This is a made up table of the Job Status types in SQL Agent:

0 - Failed

1 - Succeeded

2 - Retry

3 - Canceled

4 - Running

In [0]:
let
    Source = Table.FromList({
        [status_id=0,JobStatus="Failed"],
        [status_id=1,JobStatus="Succeeded"],
        [status_id=2,JobStatus="Retry"],
        [status_id=3,JobStatus="Canceled"],
        [status_id=4,JobStatus="Running"]
    } , Record.FieldValues, {"status_id", "Job Status"}),
    #"Changed Type" = Table.TransformColumnTypes(Source,{{"status_id", Int64.Type}, {"Job Status", type text}})
in
    #"Changed Type"

### Job Steps

SQL Query

In [0]:
SELECT [j].[Job_id]
     , [j].[name] AS [Job Name]
	 , [j].[job_order]
	 , [js].[step_name] + N' (' + CAST([j].[job_order] AS VARCHAR) +  N')' AS [Job Step Name] /* Unique, sortable */
     , [js].[step_id] AS [Step]
	 , CAST(CAST([j].[job_order] AS VARCHAR) + '.' + CAST([js].[step_id] AS VARCHAR) AS NUMERIC(18,5)) AS [step_order]
     , [js].[step_uid]
     , [js].[step_name] AS [Step Name] /* Not necessarily unique */
     , [js].[Subsystem]
     , CASE WHEN [js].[subsystem] = 'SSIS'
		   AND LEFT([js].[command], 9) = '/ISSERVER'
           THEN SUBSTRING([js].[command]
						  , CHARINDEX('"\"', [js].[command], 0) + 3
						  , CHARINDEX('""', [js].[command], 0) - CHARINDEX('"\"', [js].[command], 0) -4
						  )
			WHEN [js].[subsystem] = 'SSIS'
		   AND LEFT([js].[command], 4) = '/SQL'
		   THEN SUBSTRING([js].[command]
						  , CHARINDEX('"\', [js].[command], 0) + 1
						  , CHARINDEX('" /SERVER', [js].[command], 0) - CHARINDEX('"\"', [js].[command], 0) -7
						  )
           ELSE NULL
       END AS [Package Name]
     , CASE [js].[subsystem]
           WHEN 'SSIS'
           THEN UPPER(SUBSTRING([js].[command], CHARINDEX('/SERVER ', [js].[command], 0) + 8 --start
					 , CHARINDEX(' ', [js].[command], CHARINDEX('/SERVER ', [js].[command], 0) + 9)
                - CHARINDEX('/SERVER ', [js].[command], 0)
                - 8) --length
              )
           ELSE NULL
       END AS [Package Server]
     , [js].[Command]
     , [js].[database_name] as [DB Name]
     , [js].[database_user_name] as [DB User Name]
     , [js].[Last_Run_Outcome] as [Last Outcome]
     , [js].[Last_Run_Duration] as [Last Run Duration]
     , ( ( [js].[Last_Run_Duration] / 10000.0 * 3600
           + ( [js].[Last_Run_Duration] / 100 ) % 100 * 60
           + [js].[Last_Run_Duration] % 100 + 31 )
         / 60 ) AS [Last Run Duration Minutes]
     , CASE
           WHEN [js].[Last_Run_Duration] < 60
           THEN [js].[Last_Run_Duration]
           ELSE ( ( [js].[Last_Run_Duration] / 10000 * 3600
                    + ( [js].[Last_Run_Duration] / 100 ) % 100 * 60
                    + [js].[Last_Run_Duration] % 100 
				) )
       END AS [Last Run Duration (Sec)]
     , [js].[Retry_Attempts] as [Retry Attempts]
     , [js].[Retry_Interval] as [Retry Interval]
     , [js].[os_run_priority] as [OS Priority]
     , [js].[output_file_name] as [Output File]
       --	 , [js].[Last_Run_Retries]
     , [LR].[Last Run Datetime]
     , [js].[Last_Run_Date] AS [Last_Run_DateKey]
     , [LR].[Last Run Date]
     , [js].[Last_Run_Time] AS [Last_Run_TimeKey]
     , [LR].[Last Run Time]
FROM (SELECT [job_id]
			,[name] 
			,ROW_NUMBER() OVER (ORDER BY [name]) AS job_order
	from [msdb].[dbo].[sysjobs]  WITH(NOLOCK)) AS [J]
     INNER JOIN [msdb].[dbo].[sysjobsteps] AS [js] WITH(NOLOCK)
     ON [j].[job_id] = [js].[job_id]
     OUTER APPLY
     (
         SELECT CASE
                    WHEN [Last_run_outcome] = 1
                    THEN CAST(CONVERT(NVARCHAR(4), [js].[last_run_date] / 10000)
                              + N'-' + CONVERT(NVARCHAR(2), ( [js].[last_run_date] % 10000 ) / 100)
                              + N'-' + CONVERT(NVARCHAR(2), [js].[last_run_date] % 100)
                              + N' ' + CONVERT(NVARCHAR(2), [js].[last_run_time] / 10000)
                              + N':' + CONVERT(NVARCHAR(2), ( [js].[last_run_time] % 10000 ) / 100)
                              + N':' + CONVERT(NVARCHAR(2), [js].[last_run_time] % 100) AS DATETIME)
                    ELSE NULL
                END AS [Last Run Datetime]
              , CASE
                    WHEN [Last_run_outcome] = 1
                    THEN CAST(CONVERT(NVARCHAR(4), [js].[last_run_date] / 10000)
                              + N'-' + CONVERT(NVARCHAR(2), ( [js].[last_run_date] % 10000 ) / 100)
                              + N'-' + CONVERT(NVARCHAR(2), [js].[last_run_date] % 100) AS DATE)
                    ELSE NULL
                END AS [Last Run Date]
              , CASE
                    WHEN [Last_run_outcome] = 1
                    THEN CAST(CONVERT(NVARCHAR(2), [js].[last_run_time] / 10000)
                              + N':' + CONVERT(NVARCHAR(2), ( [js].[last_run_time] % 10000 ) / 100)
                              + N':' + CONVERT(NVARCHAR(2), [js].[last_run_time] % 100) AS TIME)
                    ELSE NULL
                END AS [Last Run Time]
     ) AS [LR]

Job Steps - M Script

In [0]:
let
    Source = Sql.Database(SQLServerInstance, "msdb", [Query="SELECT [j].[Job_id]#(lf)     , [j].[name] AS [Job Name]#(lf)#(tab) , [j].[job_order]#(lf)#(tab) , [js].[step_name] + N' (' + CAST([j].[job_order] AS VARCHAR) +  N')' AS [Job Step Name] /* Unique, sortable */#(lf)     , [js].[step_id] AS [Step]#(lf)#(tab) , CAST(CAST([j].[job_order] AS VARCHAR) + '.' + CAST([js].[step_id] AS VARCHAR) AS NUMERIC(18,5)) AS [step_order]#(lf)     , [js].[step_uid]#(lf)     , [js].[step_name] AS [Step Name] /* Not necessarily unique */#(lf)     , [js].[Subsystem]#(lf)     , CASE WHEN [js].[subsystem] = 'SSIS'#(lf)#(tab)#(tab)   AND LEFT([js].[command], 9) = '/ISSERVER'#(lf)           THEN SUBSTRING([js].[command]#(lf)#(tab)#(tab)#(tab)#(tab)#(tab)#(tab)  , CHARINDEX('""\""', [js].[command], 0) + 3#(lf)#(tab)#(tab)#(tab)#(tab)#(tab)#(tab)  , CHARINDEX('""""', [js].[command], 0) - CHARINDEX('""\""', [js].[command], 0) -4#(lf)#(tab)#(tab)#(tab)#(tab)#(tab)#(tab)  )#(lf)#(tab)#(tab)#(tab)WHEN [js].[subsystem] = 'SSIS'#(lf)#(tab)#(tab)   AND LEFT([js].[command], 4) = '/SQL'#(lf)#(tab)#(tab)   THEN SUBSTRING([js].[command]#(lf)#(tab)#(tab)#(tab)#(tab)#(tab)#(tab)  , CHARINDEX('""\', [js].[command], 0) + 1#(lf)#(tab)#(tab)#(tab)#(tab)#(tab)#(tab)  , CHARINDEX('"" /SERVER', [js].[command], 0) - CHARINDEX('""\""', [js].[command], 0) -7#(lf)#(tab)#(tab)#(tab)#(tab)#(tab)#(tab)  )#(lf)           ELSE NULL#(lf)       END AS [Package Name]#(lf)     , CASE [js].[subsystem]#(lf)           WHEN 'SSIS'#(lf)           THEN UPPER(SUBSTRING([js].[command], CHARINDEX('/SERVER ', [js].[command], 0) + 8 --start#(lf)#(tab)#(tab)#(tab)#(tab)#(tab) , CHARINDEX(' ', [js].[command], CHARINDEX('/SERVER ', [js].[command], 0) + 9)#(lf)                - CHARINDEX('/SERVER ', [js].[command], 0)#(lf)                - 8) --length#(lf)              )#(lf)           ELSE NULL#(lf)       END AS [Package Server]#(lf)     , [js].[Command]#(lf)     , [js].[database_name] as [DB Name]#(lf)     , [js].[database_user_name] as [DB User Name]#(lf)     , [js].[Last_Run_Outcome] as [Last Outcome]#(lf)     , [js].[Last_Run_Duration] as [Last Run Duration]#(lf)     , ( ( [js].[Last_Run_Duration] / 10000.0 * 3600#(lf)           + ( [js].[Last_Run_Duration] / 100 ) % 100 * 60#(lf)           + [js].[Last_Run_Duration] % 100 + 31 )#(lf)         / 60 ) AS [Last Run Duration Minutes]#(lf)     , CASE#(lf)           WHEN [js].[Last_Run_Duration] < 60#(lf)           THEN [js].[Last_Run_Duration]#(lf)           ELSE ( ( [js].[Last_Run_Duration] / 10000 * 3600#(lf)                    + ( [js].[Last_Run_Duration] / 100 ) % 100 * 60#(lf)                    + [js].[Last_Run_Duration] % 100 #(lf)#(tab)#(tab)#(tab)#(tab)) )#(lf)       END AS [Last Run Duration (Sec)]#(lf)     , [js].[Retry_Attempts] as [Retry Attempts]#(lf)     , [js].[Retry_Interval] as [Retry Interval]#(lf)     , [js].[os_run_priority] as [OS Priority]#(lf)     , [js].[output_file_name] as [Output File]#(lf)       --#(tab) , [js].[Last_Run_Retries]#(lf)     , [LR].[Last Run Datetime]#(lf)     , [js].[Last_Run_Date] AS [Last_Run_DateKey]#(lf)     , [LR].[Last Run Date]#(lf)     , [js].[Last_Run_Time] AS [Last_Run_TimeKey]#(lf)     , [LR].[Last Run Time]#(lf)FROM (SELECT [job_id]#(lf)#(tab)#(tab)#(tab),[name] #(lf)#(tab)#(tab)#(tab),ROW_NUMBER() OVER (ORDER BY [name]) AS job_order#(lf)#(tab)from [msdb].[dbo].[sysjobs]  WITH(NOLOCK)) AS [J]#(lf)     INNER JOIN [msdb].[dbo].[sysjobsteps] AS [js] WITH(NOLOCK)#(lf)     ON [j].[job_id] = [js].[job_id]#(lf)     OUTER APPLY#(lf)     (#(lf)         SELECT CASE#(lf)                    WHEN [Last_run_outcome] = 1#(lf)                    THEN CAST(CONVERT(NVARCHAR(4), [js].[last_run_date] / 10000)#(lf)                              + N'-' + CONVERT(NVARCHAR(2), ( [js].[last_run_date] % 10000 ) / 100)#(lf)                              + N'-' + CONVERT(NVARCHAR(2), [js].[last_run_date] % 100)#(lf)                              + N' ' + CONVERT(NVARCHAR(2), [js].[last_run_time] / 10000)#(lf)                              + N':' + CONVERT(NVARCHAR(2), ( [js].[last_run_time] % 10000 ) / 100)#(lf)                              + N':' + CONVERT(NVARCHAR(2), [js].[last_run_time] % 100) AS DATETIME)#(lf)                    ELSE NULL#(lf)                END AS [Last Run Datetime]#(lf)              , CASE#(lf)                    WHEN [Last_run_outcome] = 1#(lf)                    THEN CAST(CONVERT(NVARCHAR(4), [js].[last_run_date] / 10000)#(lf)                              + N'-' + CONVERT(NVARCHAR(2), ( [js].[last_run_date] % 10000 ) / 100)#(lf)                              + N'-' + CONVERT(NVARCHAR(2), [js].[last_run_date] % 100) AS DATE)#(lf)                    ELSE NULL#(lf)                END AS [Last Run Date]#(lf)              , CASE#(lf)                    WHEN [Last_run_outcome] = 1#(lf)                    THEN CAST(CONVERT(NVARCHAR(2), [js].[last_run_time] / 10000)#(lf)                              + N':' + CONVERT(NVARCHAR(2), ( [js].[last_run_time] % 10000 ) / 100)#(lf)                              + N':' + CONVERT(NVARCHAR(2), [js].[last_run_time] % 100) AS TIME)#(lf)                    ELSE NULL#(lf)                END AS [Last Run Time]#(lf)     ) AS [LR]", CreateNavigationProperties=false]),
    #"Split Column by Delimiter" = Table.SplitColumn(Source, "Package Name", Splitter.SplitTextByEachDelimiter({"\"}, QuoteStyle.Csv, true), {"Package Path", "Package Name"}),
    #"Changed Type" = Table.TransformColumnTypes(#"Split Column by Delimiter",{{"Package Path", type text}, {"Package Name", type text}})
in
    #"Changed Type"

### Job Step History


In [0]:
SELECT [jh].[job_id]
     , [jh].[step_id]
     , [js].[step_Uid]
     , [jh].[run_status]
     , [jh].[run_date]
     , [jh].[run_time]
	 , CAST(CONVERT(NVARCHAR(4), [jh].[run_date] / 10000)
            + N'-' + CONVERT(NVARCHAR(2), ( [jh].[run_date] % 10000 ) / 100)
            + N'-' + CONVERT(NVARCHAR(2), [jh].[run_date] % 100)
            + N' ' + CONVERT(NVARCHAR(2), [jh].[run_time] / 10000)
            + N':' + CONVERT(NVARCHAR(2), ( [jh].[run_time] % 10000 ) / 100)
            + N':' + CONVERT(NVARCHAR(2), [jh].[run_time] % 100) AS DATETIME) AS [Step_start_datetime]
     , DATEADD([SECOND],
               CASE
                   WHEN [jh].[run_duration] < 60
                   THEN [jh].[run_duration]
                   ELSE ( ( [jh].[run_duration] / 10000 * 3600 
						    + ( [jh].[run_duration] / 100 ) % 100 * 60
                            + [jh].[run_duration] % 100 
						) )
               END, CAST(CONVERT(NVARCHAR(4), [jh].[run_date] / 10000)
                         + N'-' + CONVERT(NVARCHAR(2), ( [jh].[run_date] % 10000 ) / 100)
                         + N'-' + CONVERT(NVARCHAR(2), [jh].[run_date] % 100)
                         + N' ' + CONVERT(NVARCHAR(2), [jh].[run_time] / 10000)
                         + N':' + CONVERT(NVARCHAR(2), ( [jh].[run_time] % 10000 ) / 100)
                         + N':' + CONVERT(NVARCHAR(2), [jh].[run_time] % 100) AS DATETIME)) AS [Step_end_datetime]
FROM [msdb].[dbo].[sysjobhistory] AS [jh]  WITH(NOLOCK)
INNER JOIN [msdb].[dbo].[sysjobsteps] AS [js] WITH(NOLOCK)
     ON [jh].[job_id] = [js].[job_id]
       AND [jh].[step_id] = [js].[step_id]
where [jh].[step_id] > 0


Job Step History - M code


In [0]:
let
    Source = Sql.Database(SQLServerInstance, "MSDB", [Query="SELECT [jh].[job_id]#(lf)     , [jh].[step_id]#(lf)     , [js].[step_Uid]#(lf)     , [jh].[run_status]#(lf)     , [jh].[run_date]#(lf)     , [jh].[run_time]#(lf)#(tab) , CAST(CONVERT(NVARCHAR(4), [jh].[run_date] / 10000)#(lf)            + N'-' + CONVERT(NVARCHAR(2), ( [jh].[run_date] % 10000 ) / 100)#(lf)            + N'-' + CONVERT(NVARCHAR(2), [jh].[run_date] % 100)#(lf)            + N' ' + CONVERT(NVARCHAR(2), [jh].[run_time] / 10000)#(lf)            + N':' + CONVERT(NVARCHAR(2), ( [jh].[run_time] % 10000 ) / 100)#(lf)            + N':' + CONVERT(NVARCHAR(2), [jh].[run_time] % 100) AS DATETIME) AS [Step_start_datetime]#(lf)     , DATEADD([SECOND],#(lf)               CASE#(lf)                   WHEN [jh].[run_duration] < 60#(lf)                   THEN [jh].[run_duration]#(lf)                   ELSE ( ( [jh].[run_duration] / 10000 * 3600 #(lf)#(tab)#(tab)#(tab)#(tab)#(tab)#(tab)    + ( [jh].[run_duration] / 100 ) % 100 * 60#(lf)                            + [jh].[run_duration] % 100 #(lf)#(tab)#(tab)#(tab)#(tab)#(tab)#(tab)) )#(lf)               END, CAST(CONVERT(NVARCHAR(4), [jh].[run_date] / 10000)#(lf)                         + N'-' + CONVERT(NVARCHAR(2), ( [jh].[run_date] % 10000 ) / 100)#(lf)                         + N'-' + CONVERT(NVARCHAR(2), [jh].[run_date] % 100)#(lf)                         + N' ' + CONVERT(NVARCHAR(2), [jh].[run_time] / 10000)#(lf)                         + N':' + CONVERT(NVARCHAR(2), ( [jh].[run_time] % 10000 ) / 100)#(lf)                         + N':' + CONVERT(NVARCHAR(2), [jh].[run_time] % 100) AS DATETIME)) AS [Step_end_datetime]#(lf)FROM [msdb].[dbo].[sysjobhistory] AS [jh]  WITH(NOLOCK)#(lf)INNER JOIN [msdb].[dbo].[sysjobsteps] AS [js] WITH(NOLOCK)#(lf)     ON [jh].[job_id] = [js].[job_id]#(lf)       AND [jh].[step_id] = [js].[step_id]#(lf)where [jh].[step_id] > 0#(lf)", CreateNavigationProperties=false]),
    #"Add Duration in Seconds" = Table.AddColumn(Source, "Step_Duration_Seconds", each Duration.TotalSeconds([Step_end_datetime] - [Step_start_datetime])),
    #"Changed Type" = Table.TransformColumnTypes(#"Add Duration in Seconds",{{"Step_Duration_Seconds", Int64.Type}})
in
    #"Changed Type"

### Schedules

In [0]:
DECLARE @weekDay TABLE
([BitMask] INT, [maskValue] VARCHAR(32));

INSERT INTO @weekDay
( [BitMask]
, [maskValue]
)
VALUES
( 1, 'Sunday' ),
( 2, 'Monday' ),
( 4, 'Tuesday' ),
( 8, 'Wednesday' ),
( 16, 'Thursday' ),
( 32, 'Friday' ),
( 64, 'Saturday' );

WITH [CTE_Sched]
AS (
SELECT [sched].[name] AS [Schedule Name]
	, [sched].[enabled] AS [Sched Enabled]
	, [sched].[schedule_id]
	, CASE WHEN [sched].[freq_type] = 1 THEN 'Once'
		WHEN [sched].[freq_type] = 4 
		AND [sched].[freq_interval] = 1 THEN 'Daily'
		WHEN [sched].[freq_type] = 4 	THEN 'Every '
			+ CAST([sched].[freq_interval] AS VARCHAR(5)) + ' days'
		WHEN [sched].[freq_type] = 8
		THEN REPLACE(REPLACE(REPLACE(
			(	SELECT [MaskValue]
				FROM @weekDay AS [M]
				WHERE [sched].[freq_interval] & [M].[BitMask] <> 0
				ORDER BY [BitMask] FOR XML RAW
			), '"/><row maskValue="', ', '), '<row maskValue="', ''), '"/>', '')
		+ CASE
			WHEN [sched].[freq_recurrence_factor] <> 0
				AND [sched].[freq_recurrence_factor] = 1
			THEN '; weekly'
			WHEN [sched].[freq_recurrence_factor] <> 0
			THEN '; every '
				+ CAST([sched].[freq_recurrence_factor] AS VARCHAR(10))
				+ ' weeks'
		END
		WHEN [sched].[freq_type] = 16
		THEN 'On day '
		+ CAST([sched].[freq_interval] AS VARCHAR(10))
		+ ' of every '
		+ CASE [sched].[freq_recurrence_factor] 
			WHEN 1 THEN 'month'
			WHEN 2 THEN 'second month'
			WHEN 3 THEN 'third month'
			ELSE CAST([sched].[freq_recurrence_factor] AS VARCHAR(10))	+ 'th month'
		  END
		WHEN [sched].[freq_type] = 32
		THEN CASE
			WHEN [sched].[freq_relative_interval] = 1 THEN 'First'
			WHEN [sched].[freq_relative_interval] = 2 THEN 'Second'
			WHEN [sched].[freq_relative_interval] = 4 THEN 'Third'
			WHEN [sched].[freq_relative_interval] = 8 THEN 'Fourth'
			WHEN [sched].[freq_relative_interval] = 16 THEN 'Last'
		END
		+ CASE
			WHEN [sched].[freq_interval] = 1 THEN ' Sunday'
			WHEN [sched].[freq_interval] = 2 THEN ' Monday'
			WHEN [sched].[freq_interval] = 3 THEN ' Tuesday'
			WHEN [sched].[freq_interval] = 4 THEN ' Wednesday'
			WHEN [sched].[freq_interval] = 5 THEN ' Thursday'
			WHEN [sched].[freq_interval] = 6 THEN ' Friday'
			WHEN [sched].[freq_interval] = 7 THEN ' Saturday'
			WHEN [sched].[freq_interval] = 8 THEN ' Day'
			WHEN [sched].[freq_interval] = 9 THEN ' Weekday'
			WHEN [sched].[freq_interval] = 10 THEN ' Weekend'
		END
		+ CASE
			WHEN [sched].[freq_recurrence_factor] <> 0
				AND [sched].[freq_recurrence_factor] = 1
			THEN '; monthly'
			WHEN [sched].[freq_recurrence_factor] <> 0
			THEN '; every '
				+ CAST([sched].[freq_recurrence_factor] AS VARCHAR(10))
				+ ' months'
		END
		WHEN [sched].[freq_type] = 64 THEN 'StartUp'
		WHEN [sched].[freq_type] = 128 THEN 'Idle'
	  END AS [Frequency]
	, ISNULL('Every '
	  + CAST([sched].[freq_subday_interval] AS VARCHAR(10))
	  + CASE
		WHEN [sched].[freq_subday_type] = 2 THEN ' seconds'
		WHEN [sched].[freq_subday_type] = 4 THEN ' minutes'
		WHEN [sched].[freq_subday_type] = 8 THEN ' hours'
	  END, 'Once') AS [SubFrequency]
	, REPLICATE('0', 6 - LEN([sched].[active_start_time]))
	  + CAST([sched].[active_start_time] AS VARCHAR(6)) AS [start_time]
	, REPLICATE('0', 6 - LEN([sched].[active_end_time]))
	  + CAST([sched].[active_end_time] AS VARCHAR(6)) AS [end_time]
FROM [dbo].[sysschedules] AS [sched]
)
SELECT [schedule_id]
	,[Schedule Name]
	,[Sched Enabled]
	,[Frequency]
	,[SubFrequency]
	,CAST([start_time] AS INT) AS [start_timekey]
	, CAST(CONVERT(NVARCHAR(2), [start_time] / 10000)
		+ N':'+ CONVERT(NVARCHAR(2), ( [start_time] % 10000 ) / 100)
		+ N':'+ CONVERT(NVARCHAR(2),   [start_time] % 100) AS TIME) AS [Sched Start Time]
	,CASE WHEN [SubFrequency] = 'Once'
		THEN DATEADD(Minute, 1, CAST(CONVERT(NVARCHAR(2), [start_time] / 10000)
					+ N':'+ CONVERT(NVARCHAR(2), ( [start_time] % 10000 ) / 100)
					+ N':'+ CONVERT(NVARCHAR(2),   [start_time] % 100) AS TIME))
		ELSE CAST(CONVERT(NVARCHAR(2),   [end_time] / 10000)
		+ N':'+ CONVERT(NVARCHAR(2), ( [end_time] % 10000 ) / 100)
		+ N':'+ CONVERT(NVARCHAR(2),   [end_time] % 100) AS TIME) 
	END AS [Sched End Time]
from [CTE_Sched] 

Add a step to the Schedule query change the "Sched Enabled" column to a type of Logical:

In [0]:
let
    Source = Sql.Database(SQLServerInstance, "msdb", [Query="DECLARE @weekDay TABLE#(lf)([BitMask] INT, [maskValue] VARCHAR(32));#(lf)#(lf)INSERT INTO @weekDay#(lf)( [BitMask]#(lf), [maskValue]#(lf))#(lf)VALUES#(lf)( 1, 'Sunday' ),#(lf)( 2, 'Monday' ),#(lf)( 4, 'Tuesday' ),#(lf)( 8, 'Wednesday' ),#(lf)( 16, 'Thursday' ),#(lf)( 32, 'Friday' ),#(lf)( 64, 'Saturday' );#(lf)#(lf)WITH [CTE_Sched]#(lf)AS (#(lf)SELECT [sched].[name] AS [Schedule Name]#(lf)#(tab), [sched].[enabled] AS [Sched Enabled]#(lf)#(tab), [sched].[schedule_id]#(lf)#(tab), CASE WHEN [sched].[freq_type] = 1 THEN 'Once'#(lf)#(tab)#(tab)WHEN [sched].[freq_type] = 4 #(lf)#(tab)#(tab)AND [sched].[freq_interval] = 1 THEN 'Daily'#(lf)#(tab)#(tab)WHEN [sched].[freq_type] = 4 #(tab)THEN 'Every '#(lf)#(tab)#(tab)#(tab)+ CAST([sched].[freq_interval] AS VARCHAR(5)) + ' days'#(lf)#(tab)#(tab)WHEN [sched].[freq_type] = 8#(lf)#(tab)#(tab)THEN REPLACE(REPLACE(REPLACE(#(lf)#(tab)#(tab)#(tab)(#(tab)SELECT [MaskValue]#(lf)#(tab)#(tab)#(tab)#(tab)FROM @weekDay AS [M]#(lf)#(tab)#(tab)#(tab)#(tab)WHERE [sched].[freq_interval] & [M].[BitMask] <> 0#(lf)#(tab)#(tab)#(tab)#(tab)ORDER BY [BitMask] FOR XML RAW#(lf)#(tab)#(tab)#(tab)), '""/><row maskValue=""', ', '), '<row maskValue=""', ''), '""/>', '')#(lf)#(tab)#(tab)+ CASE#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_recurrence_factor] <> 0#(lf)#(tab)#(tab)#(tab)#(tab)AND [sched].[freq_recurrence_factor] = 1#(lf)#(tab)#(tab)#(tab)THEN '; weekly'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_recurrence_factor] <> 0#(lf)#(tab)#(tab)#(tab)THEN '; every '#(lf)#(tab)#(tab)#(tab)#(tab)+ CAST([sched].[freq_recurrence_factor] AS VARCHAR(10))#(lf)#(tab)#(tab)#(tab)#(tab)+ ' weeks'#(lf)#(tab)#(tab)END#(lf)#(tab)#(tab)WHEN [sched].[freq_type] = 16#(lf)#(tab)#(tab)THEN 'On day '#(lf)#(tab)#(tab)+ CAST([sched].[freq_interval] AS VARCHAR(10))#(lf)#(tab)#(tab)+ ' of every '#(lf)#(tab)#(tab)+ CASE [sched].[freq_recurrence_factor] #(lf)#(tab)#(tab)#(tab)WHEN 1 THEN 'month'#(lf)#(tab)#(tab)#(tab)WHEN 2 THEN 'second month'#(lf)#(tab)#(tab)#(tab)WHEN 3 THEN 'third month'#(lf)#(tab)#(tab)#(tab)ELSE CAST([sched].[freq_recurrence_factor] AS VARCHAR(10))#(tab)+ 'th month'#(lf)#(tab)#(tab)  END#(lf)#(tab)#(tab)WHEN [sched].[freq_type] = 32#(lf)#(tab)#(tab)THEN CASE#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_relative_interval] = 1 THEN 'First'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_relative_interval] = 2 THEN 'Second'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_relative_interval] = 4 THEN 'Third'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_relative_interval] = 8 THEN 'Fourth'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_relative_interval] = 16 THEN 'Last'#(lf)#(tab)#(tab)END#(lf)#(tab)#(tab)+ CASE#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_interval] = 1 THEN ' Sunday'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_interval] = 2 THEN ' Monday'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_interval] = 3 THEN ' Tuesday'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_interval] = 4 THEN ' Wednesday'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_interval] = 5 THEN ' Thursday'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_interval] = 6 THEN ' Friday'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_interval] = 7 THEN ' Saturday'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_interval] = 8 THEN ' Day'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_interval] = 9 THEN ' Weekday'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_interval] = 10 THEN ' Weekend'#(lf)#(tab)#(tab)END#(lf)#(tab)#(tab)+ CASE#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_recurrence_factor] <> 0#(lf)#(tab)#(tab)#(tab)#(tab)AND [sched].[freq_recurrence_factor] = 1#(lf)#(tab)#(tab)#(tab)THEN '; monthly'#(lf)#(tab)#(tab)#(tab)WHEN [sched].[freq_recurrence_factor] <> 0#(lf)#(tab)#(tab)#(tab)THEN '; every '#(lf)#(tab)#(tab)#(tab)#(tab)+ CAST([sched].[freq_recurrence_factor] AS VARCHAR(10))#(lf)#(tab)#(tab)#(tab)#(tab)+ ' months'#(lf)#(tab)#(tab)END#(lf)#(tab)#(tab)WHEN [sched].[freq_type] = 64 THEN 'StartUp'#(lf)#(tab)#(tab)WHEN [sched].[freq_type] = 128 THEN 'Idle'#(lf)#(tab)  END AS [Frequency]#(lf)#(tab), ISNULL('Every '#(lf)#(tab)  + CAST([sched].[freq_subday_interval] AS VARCHAR(10))#(lf)#(tab)  + CASE#(lf)#(tab)#(tab)WHEN [sched].[freq_subday_type] = 2 THEN ' seconds'#(lf)#(tab)#(tab)WHEN [sched].[freq_subday_type] = 4 THEN ' minutes'#(lf)#(tab)#(tab)WHEN [sched].[freq_subday_type] = 8 THEN ' hours'#(lf)#(tab)  END, 'Once') AS [SubFrequency]#(lf)#(tab), REPLICATE('0', 6 - LEN([sched].[active_start_time]))#(lf)#(tab)  + CAST([sched].[active_start_time] AS VARCHAR(6)) AS [start_time]#(lf)#(tab), REPLICATE('0', 6 - LEN([sched].[active_end_time]))#(lf)#(tab)  + CAST([sched].[active_end_time] AS VARCHAR(6)) AS [end_time]#(lf)FROM [dbo].[sysschedules] AS [sched]#(lf))#(lf)SELECT [schedule_id]#(lf)#(tab),[Schedule Name]#(lf)#(tab),[Sched Enabled]#(lf)#(tab),[Frequency]#(lf)#(tab),[SubFrequency]#(lf)#(tab),CAST([start_time] AS INT) AS [start_timekey]#(lf)#(tab), CAST(CONVERT(NVARCHAR(2), [start_time] / 10000)#(lf)#(tab)#(tab)+ N':'+ CONVERT(NVARCHAR(2), ( [start_time] % 10000 ) / 100)#(lf)#(tab)#(tab)+ N':'+ CONVERT(NVARCHAR(2),   [start_time] % 100) AS TIME) AS [Sched Start Time]#(lf)#(tab),CASE WHEN [SubFrequency] = 'Once'#(lf)#(tab)#(tab)THEN DATEADD(Minute, 1, CAST(CONVERT(NVARCHAR(2), [start_time] / 10000)#(lf)#(tab)#(tab)#(tab)#(tab)#(tab)+ N':'+ CONVERT(NVARCHAR(2), ( [start_time] % 10000 ) / 100)#(lf)#(tab)#(tab)#(tab)#(tab)#(tab)+ N':'+ CONVERT(NVARCHAR(2),   [start_time] % 100) AS TIME))#(lf)#(tab)#(tab)ELSE CAST(CONVERT(NVARCHAR(2),   [end_time] / 10000)#(lf)#(tab)#(tab)+ N':'+ CONVERT(NVARCHAR(2), ( [end_time] % 10000 ) / 100)#(lf)#(tab)#(tab)+ N':'+ CONVERT(NVARCHAR(2),   [end_time] % 100) AS TIME) #(lf)#(tab)END AS [Sched End Time]#(lf)from [CTE_Sched] ", CreateNavigationProperties=false]),
    #"Changed Type" = Table.TransformColumnTypes(Source,{{"Sched Enabled", type logical}})
in
    #"Changed Type"

## Other Useful queries:

- RefreshTime - Display in reports to show when the data was last refreshed

In [0]:
let
    Source = DateTimeZone.UtcNow(),
    #"Converted to Table" = #table(1, {{Source}}),
    #"Added Custom" = Table.AddColumn(#"Converted to Table", "CST", each DateTimeZone.SwitchZone([Column1],-6)),
    #"Renamed Columns" = Table.RenameColumns(#"Added Custom",{{"Column1", "Refresh Time UTC"}, {"CST", "Refresh Time CST"}}),
    #"Changed Type" = Table.TransformColumnTypes(#"Renamed Columns",{{"Refresh Time CST", type datetime}, {"Refresh Time UTC", type datetime}}),
    #"Changed Type1" = Table.TransformColumnTypes(#"Changed Type",{{"Refresh Time UTC", type datetimezone}})
in
    #"Changed Type1"

## Wiring up the model

Table relationships needed:

> Alerts(category_id) <- Job Categories(category_id)

> Job(category_id) <- Job Categories(category_id)

> job clash(TimeKey) <- Time of Day(TimeKey)

> Job History(job_id) <- Job(job_id)

> Job History(run_date) <- Calendar(DateKey)

> Job History(run_status) <- Job Status(status_id)

> Job History(run_time) <- Time of Day(TimeKey)

> Job Schedules(Job_id) <- Job(job_id)

> Job Schedules(next_run_datekey) <- Calendar(DateKey)

> Job Schedules(schedule_id) <- Schedules(schedule_id)

> Job Step History(run_date) <- Calendar(DateKey) 

> Job Step History(run_status) <- Job Status(status_id)

> Job Step History(run_time) <- Time of Day(TimeKey)

> Job Step History(step_Uid) <- Job Steps (step_uid)

> Job Steps(Job_id) <- Job(Job_id) 

> Job Steps(Last_Run_DateKey) <- Calendar(DateKey) --inactive

> Job Steps(Last_Run_TimeKey)  <- Time of Day(TimeKey) --inactive

> Report Jobs(SQLAgent_Job_Name) <- Job(Job Name)

> Schedules(start_timekey) <= Time of Day(TimeKey)

## DAX calculated columns

### Table: Schedules

> Sched Full Name = [Schedule Name] & " (" & [schedule_id] & ")" 

> Dependent Job count = SUMX(RELATEDTABLE('Job Schedules'), [Scheduled Jobs]) 

### Table: SQL Agent Settings

> Job History Table Limit  = IF('SQLAgent Settings'[MaxJobHistoryTableRows] = -1, "Unlimited", CONCATENATE('SQLAgent Settings'[MaxJobHistoryTableRows], " Rows")) 

> Job History Limit = IF('SQLAgent Settings'[MaxJobHistoryRows] = 0, "Unlimited", CONCATENATE('SQLAgent Settings'[MaxJobHistoryRows], " Rows")) 

## DAX Calculations

Note: the calculations are listed per table, but could also be created under and emtpy "measures" table instead of under each specified table.





In [0]:
/*Job table*/
Job Count = COUNTROWS('Job')

Last Run Duration (sec) = sum('Job'[Last_Run_Duration])

Last Run Duration =
// Duration formatting : Given a number of seconds, returns a format of "n days, hh:mm:ss" 
// We start with a duration in number of seconds VAR Duration = SUM ( 'Job'[Last_Run_Duration] ) 
VAR Days =
    INT ( Duration / 86400 )
VAR Hours =
    INT ( MOD ( Duration - ( Days * 86400 ), 86400 ) / 3600 )
VAR Minutes =
    INT ( MOD ( Duration - ( Hours * 3600 ), 3600 ) / 60 )
VAR Seconds =
    ROUNDUP ( MOD ( MOD ( Duration - ( Hours * 3600 ), 3600 ), 60 ), 0 ) // We round up here to get a whole number 
// Day or Days 
VAR D =
    IF (
        Days = 1,
        CONCATENATE ( Days, " day " ),
        IF ( Days > 1, CONCATENATE ( Days, " days " ), "" )
    ) // Hours with leading zeros 
VAR H =
    IF ( LEN ( Hours ) = 1, CONCATENATE ( "0", Hours ), CONCATENATE ( "", Hours ) ) // Minutes with leading zeros 
VAR M =
    IF (
        LEN ( Minutes ) = 1,
        CONCATENATE ( "0", Minutes ),
        CONCATENATE ( "", Minutes )
    ) // Seconds with leading zeros 
VAR S =
    IF (
        LEN ( Seconds ) = 1,
        CONCATENATE ( "0", Seconds ),
        CONCATENATE ( "", Seconds )
    ) // Now return hours, minutes and seconds with leading zeros in the proper format "hh:mm:ss" 
RETURN
    CONCATENATE (
        D,
        CONCATENATE (
            H,
            CONCATENATE ( ":", CONCATENATE ( M, CONCATENATE ( ":", S ) ) )
        )
    )

/* Job History table */
Job Duration (Sec) = sum([Job_Duration_Seconds])

Duration Avg (Sec) = AVERAGE('Job History'[Job_Duration_Seconds])

Job Exec Count = countrows('Job History')
Duration =
// Duration formatting : Given a number of seconds, returns a format of "n days, hh:mm:ss" 
// We start with a duration in number of seconds 
VAR Duration =
    SUM ( 'Job History'[Job_Duration_Seconds] )
VAR Days =
    INT ( Duration / 86400 )
VAR Hours =
    INT ( MOD ( Duration - ( Days * 86400 ), 86400 ) / 3600 )
VAR Minutes =
    INT ( MOD ( Duration - ( Hours * 3600 ), 3600 ) / 60 )
VAR Seconds =
    ROUNDUP ( MOD ( MOD ( Duration - ( Hours * 3600 ), 3600 ), 60 ), 0 ) // We round up here to get a whole number 
// Day or Days 
VAR D =
    IF (
        Days = 1,
        CONCATENATE ( Days, " day " ),
        IF ( Days > 1, CONCATENATE ( Days, " days " ), "" )
    ) // Hours with leading zeros 
VAR H =
    IF ( LEN ( Hours ) = 1, CONCATENATE ( "0", Hours ), CONCATENATE ( "", Hours ) ) // Minutes with leading zeros 
VAR M =
    IF (
        LEN ( Minutes ) = 1,
        CONCATENATE ( "0", Minutes ),
        CONCATENATE ( "", Minutes )
    ) // Seconds with leading zeros 
VAR S =
    IF (
        LEN ( Seconds ) = 1,
        CONCATENATE ( "0", Seconds ),
        CONCATENATE ( "", Seconds )
    ) // Now return hours, minutes and seconds with leading zeros in the proper format "hh:mm:ss"
RETURN
    IFERROR (
        IF (
            Duration = 0,
            BLANK (),
            CONCATENATE (
                D,
                CONCATENATE (
                    H,
                    CONCATENATE ( ":", CONCATENATE ( M, CONCATENATE ( ":", S ) ) )
                )
            )
        ),
        BLANK ()
    )

Duration Avg =
// Duration formatting : Given a number of seconds, returns a format of "n days, hh:mm:ss" 
// We start with a duration in number of seconds 
VAR Duration =
    AVERAGE ( 'Job History'[Job_Duration_Seconds] )
VAR Days =
    INT ( Duration / 86400 )
VAR Hours =
    INT ( MOD ( Duration - ( Days * 86400 ), 86400 ) / 3600 )
VAR Minutes =
    INT ( MOD ( Duration - ( Hours * 3600 ), 3600 ) / 60 )
VAR Seconds =
    ROUNDUP ( MOD ( MOD ( Duration - ( Hours * 3600 ), 3600 ), 60 ), 0 ) // We round up here to get a whole number 
// Day or Days 
VAR D =
    IF (
        Days = 1,
        CONCATENATE ( Days, " day " ),
        IF ( Days > 1, CONCATENATE ( Days, " days " ), "" )
    ) // Hours with leading zeros 
VAR H =
    IF ( LEN ( Hours ) = 1, CONCATENATE ( "0", Hours ), CONCATENATE ( "", Hours ) ) // Minutes with leading zeros 
VAR M =
    IF (
        LEN ( Minutes ) = 1,
        CONCATENATE ( "0", Minutes ),
        CONCATENATE ( "", Minutes )
    ) // Seconds with leading zeros 
VAR S =
    IF (
        LEN ( Seconds ) = 1,
        CONCATENATE ( "0", Seconds ),
        CONCATENATE ( "", Seconds )
    ) // Now return hours, minutes and seconds with leading zeros in the proper format "hh:mm:ss" 
RETURN
    IFERROR (
        IF (
            Duration = 0,
            BLANK (),
            CONCATENATE (
                D,
                CONCATENATE (
                    H,
                    CONCATENATE ( ":", CONCATENATE ( M, CONCATENATE ( ":", S ) ) )
                )
            )
        ),
        BLANK ()
    )

Job Exec Failed =
VAR __BASELINE_VALUE =
    CALCULATE ( [Job Exec Count], 'Job Status'[Job Status] IN { "Succeeded" } )
VAR __MEASURE_VALUE = [Job Exec Count]
RETURN
    IF ( NOT ISBLANK ( __MEASURE_VALUE ), __MEASURE_VALUE - __BASELINE_VALUE )

Duration % Chg = DIVIDE( ([Duration (Last)] - [Duration (First)]), [Duration (First)] ,0)

Duration Min =
// Duration formatting : Given a number of seconds, returns a format of "n days, hh:mm:ss" 
// We start with a duration in number of seconds 
VAR Duration =
    MIN ( 'Job History'[Job_Duration_Seconds] )
VAR Days =
    INT ( Duration / 86400 )
VAR Hours =
    INT ( MOD ( Duration - ( Days * 86400 ), 86400 ) / 3600 )
VAR Minutes =
    INT ( MOD ( Duration - ( Hours * 3600 ), 3600 ) / 60 )
VAR Seconds =
    ROUNDUP ( MOD ( MOD ( Duration - ( Hours * 3600 ), 3600 ), 60 ), 0 ) // We round up here to get a whole number 
// Day or Days 
VAR D =
    IF (
        Days = 1,
        CONCATENATE ( Days, " day " ),
        IF ( Days > 1, CONCATENATE ( Days, " days " ), "" )
    ) // Hours with leading zeros 
VAR H =
    IF ( LEN ( Hours ) = 1, CONCATENATE ( "0", Hours ), CONCATENATE ( "", Hours ) ) // Minutes with leading zeros 
VAR M =
    IF (
        LEN ( Minutes ) = 1,
        CONCATENATE ( "0", Minutes ),
        CONCATENATE ( "", Minutes )
    ) // Seconds with leading zeros 
VAR S =
    IF (
        LEN ( Seconds ) = 1,
        CONCATENATE ( "0", Seconds ),
        CONCATENATE ( "", Seconds )
    ) // Now return hours, minutes and seconds with leading zeros in the proper format "hh:mm:ss" 
RETURN
    IFERROR (
        IF (
            Duration = 0,
            BLANK (),
            CONCATENATE (
                D,
                CONCATENATE (
                    H,
                    CONCATENATE ( ":", CONCATENATE ( M, CONCATENATE ( ":", S ) ) )
                )
            )
        ),
        BLANK ()
    )

Duration Max =
// Duration formatting : Given a number of seconds, returns a format of "n days, hh:mm:ss" 
// We start with a duration in number of seconds VAR Duration = MAX('Job History'[Job_Duration_Seconds]) 
VAR Days =
    INT ( Duration / 86400 )
VAR Hours =
    INT ( MOD ( Duration - ( Days * 86400 ), 86400 ) / 3600 )
VAR Minutes =
    INT ( MOD ( Duration - ( Hours * 3600 ), 3600 ) / 60 )
VAR Seconds =
    ROUNDUP ( MOD ( MOD ( Duration - ( Hours * 3600 ), 3600 ), 60 ), 0 ) // We round up here to get a whole number 
// Day or Days 
VAR D =
    IF (
        Days = 1,
        CONCATENATE ( Days, " day " ),
        IF ( Days > 1, CONCATENATE ( Days, " days " ), "" )
    ) // Hours with leading zeros 
VAR H =
    IF ( LEN ( Hours ) = 1, CONCATENATE ( "0", Hours ), CONCATENATE ( "", Hours ) ) // Minutes with leading zeros 
VAR M =
    IF (
        LEN ( Minutes ) = 1,
        CONCATENATE ( "0", Minutes ),
        CONCATENATE ( "", Minutes )
    ) // Seconds with leading zeros 
VAR S =
    IF (
        LEN ( Seconds ) = 1,
        CONCATENATE ( "0", Seconds ),
        CONCATENATE ( "", Seconds )
    ) // Now return hours, minutes and seconds with leading zeros in the proper format "hh:mm:ss" 
RETURN
    IFERROR (
        IF (
            Duration = 0,
            BLANK (),
            CONCATENATE (
                D,
                CONCATENATE (
                    H,
                    CONCATENATE ( ":", CONCATENATE ( M, CONCATENATE ( ":", S ) ) )
                )
            )
        ),
        BLANK ()
    )

Duration (Last) =
CALCULATE (
    SUM ( 'Job History'[Job_Duration_Seconds] ),
    FILTER (
        'Job History',
        'Job History'[Job_end_datetime] = MAX ( 'Job History'[Job_end_datetime] )
    )
)

Duration (First) =
CALCULATE (
    SUM ( 'Job History'[Job_Duration_Seconds] ),
    FILTER (
        'Job History',
        'Job History'[Job_end_datetime] = MIN ( 'Job History'[Job_end_datetime] )
    )
)

Duration (Total) = sum('Job History'[Job_Duration_Seconds])

Scheduled Jobs = DISTINCTCOUNT('Job Schedules'[job_id])

/* Job Clash table */
Running Jobs = sum('Job Clash'[Running])
Running Jobs (avg) = AVERAGE('Job Clash'[Running])
Concurrent Jobs = MAX('Job Clash'[Running])

/* Alerts table */
Alert Count = if(ISBLANK(COUNTROWS(Alerts)), 0 , COUNTROWS(Alerts))
Occurrence Count = IF(ISBLANK(sum(Alerts[Occurences])), 0 , sum(Alerts[Occurences]))

/* Job Step History table */
Step Duration (First) =
CALCULATE (
    SUM ( 'Job Step History'[Step_Duration_Seconds] ),
    FILTER (
        'Job Step History',
        'Job Step History'[Step_end_datetime]
            = MIN ( 'Job Step History'[Step_end_datetime] )
    )
)

Step Duration (Last) =
CALCULATE (
    SUM ( 'Job Step History'[Step_Duration_Seconds] ),
    FILTER (
        'Job Step History',
        'Job Step History'[Step_end_datetime]
            = MAX ( 'Job Step History'[Step_end_datetime] )
    )
)

Step Duration (Total) = sum('Job Step History'[Step_Duration_Seconds])

Step Duration (Sec) = sum('Job Step History'[Step_Duration_Seconds])

Step Duration =
// Duration formatting : Given a number of seconds, returns a format of "n days, hh:mm:ss" 
// We start with a duration in number of seconds 
VAR Duration =
    SUM ( 'Job Step History'[Step_Duration_Seconds] )
VAR Days =
    INT ( Duration / 86400 )
VAR Hours =
    INT ( MOD ( Duration - ( Days * 86400 ), 86400 ) / 3600 )
VAR Minutes =
    INT ( MOD ( Duration - ( Hours * 3600 ), 3600 ) / 60 )
VAR Seconds =
    ROUNDUP ( MOD ( MOD ( Duration - ( Hours * 3600 ), 3600 ), 60 ), 0 ) // We round up here to get a whole number 
// Day or Days 
VAR D =
    IF (
        Days = 1,
        CONCATENATE ( Days, " day " ),
        IF ( Days > 1, CONCATENATE ( Days, " days " ), "" )
    ) // Hours with leading zeros 
VAR H =
    IF ( LEN ( Hours ) = 1, CONCATENATE ( "0", Hours ), CONCATENATE ( "", Hours ) ) // Minutes with leading zeros 
VAR M =
    IF (
        LEN ( Minutes ) = 1,
        CONCATENATE ( "0", Minutes ),
        CONCATENATE ( "", Minutes )
    ) // Seconds with leading zeros VAR S = IF ( LEN ( Seconds ) = 1, CONCATENATE ( "0", Seconds ), CONCATENATE ( "", Seconds ) ) 
// Now return hours, minutes and seconds with leading zeros in the proper format "hh:mm:ss" 
RETURN
    IFERROR (
        IF (
            Duration = 0,
            BLANK (),
            CONCATENATE (
                D,
                CONCATENATE (
                    H,
                    CONCATENATE ( ":", CONCATENATE ( M, CONCATENATE ( ":", S ) ) )
                )
            )
        ),
        BLANK ()
    )
Step Count = DISTINCTCOUNT('Job Step History'[step_Uid])

Step Exec Count = COUNTROWS('Job Step History')

Step Exec Failed =
VAR __BASELINE_VALUE =
    CALCULATE ( [Job Exec Count], 'Job Status'[Job Status] IN { "Succeeded" } )
VAR __MEASURE_VALUE = [Step Exec Count]
RETURN
    IF ( NOT ISBLANK ( __MEASURE_VALUE ), __MEASURE_VALUE - __BASELINE_VALUE )
